### 企查查

### 1.按照红头文件匹配

In [1]:
import jieba
import pandas as pd

In [2]:
def get_stop_words():
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])

In [3]:
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']

    
def hadle_official_info():
    '''讲地市的红头文件分词'''
    df = pd.read_csv('/home/lj/Desktop/redhead.csv',delimiter='$',header=-1)
    df.head()
    df[1] = df[1].apply(lambda x:strQ2B(x))
    jieba.set_dictionary('data/dict.txt.small')
    x_cut = [] #分词后,去除停用词的语料库

    # 获取停用词列表
    stopwords = get_stop_words()
    cut_res = df[1].apply(cut)
    df[1] = cut_res
    return df

official_info_df = hadle_official_info()

Building prefix dict from /home/lj/ideaprojects/DeepLearning/data/dict.txt.small ...
Loading model from cache /tmp/jieba.ube6b3622f3ed3770dfb4b3dbec1af2a2.cache
Loading model cost 0.158 seconds.
Prefix dict has been built succesfully.


In [4]:
official_info_df

,0,1
0,济南,信息技术/智能/制造/高端/装备/量子/科技/生物/医药/先进/材料/产业/金融/现代/物流...
1,青岛,西海岸/新区/蓝谷/核心区/高新区/胶东/临空/经济/示范区/胶州湾/青岛/老城区/有机/更...
2,烟台,烟台/经济/技术开发区/烟台/高新技术/产业/开发区/烟台/东部/产城/融合/发展/示范区/...
3,淄博,国家/省级/经济/技术开发区/高新技术/产业/开发区/海关/特殊/监管/区/新/能源/电池/...
4,枣庄,人工智能/信息技术/新/能源/新/材料/医养/健康
5,东营,航空航天/石化/装备/新/能源/文化/旅游
6,潍坊,虚拟/现实/人工智能/新/能源/电池/高端/装备/制造/生物/基/材料/信息技术/现代/种业
7,济宁,第三代/半导体/生命/健康/信息技术/文化/旅游/生物/医药
8,泰安,人工智能/生命/健康/信息技术/高端/装备/制造/文化/旅游/体育/新/能源
9,威海,生命/健康/前沿/新/材料/医疗/器械/海洋生物/时尚/创意


In [5]:
official_info_df.to_csv('redhead2.csv',index=0)

In [6]:
official_info_df = pd.read_csv('redhead2.csv',index_col=0)
official_info_df.index
official_info_df.loc['青岛','1']
type(official_info_df.loc['青岛','1'])

str

In [7]:
def judge_by_official(desc,location):
    '''匹配红头文件, 存在则判定为新动能
       -1 : 未找到红头文件
       0: 匹配失败
       1: 新动能匹配成功
    '''
    try:
        print(location)
        official_info_df.loc[location]
    except KeyError: 
        return -1
    official_info = official_info_df.loc[location,'1']
    official_info = official_info.split('/')
    crossinfo = [w for w in desc if w in official_info]
    print('pattern ',location, ' ',len(crossinfo),'字符')
    if len(crossinfo)>1:
        return 1
    else:
        return 0

desc = '青岛达能环保设备股份有限公司，成立于2006年10月，注册资金6300万元，占地面积15万平方米，生产建筑面积5.5万平方米，制造设备齐全，生产装备达到国内同行业领先水平，目前已具备年产值10亿元人民币的生产能力。青岛达能环保设备股份有限公司属锅炉辅机、环保设备制造行业。主要从事电站锅炉干式除渣系统产品、湿式除渣系统产品及烟气余热利用系统产品的研发、生产与销售。青岛达能环保设备股份有限公司为集研发、生产、销售为一体的国家高新技术企业、国家AAA级信用企业、山东省战略新兴产业重点企业、青岛企业技术中心、青岛民营科技创新先进单位。获得国家民营科技发展贡献奖、国家重点新产品、山东优秀节能成果奖等项目的荣誉称号和奖励。随着公司新业务的扩展，现面向社会诚招以下人才'
desc = handle_desc(desc)
location = '青岛'
judge_by_official(desc,location)

青岛
pattern  青岛   16 字符


1

### 2. LDA匹配

In [33]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
qichacha_cut = pd.read_csv('qichacha_cut.csv',index_col=0)
x_cut = qichacha_cut['经营范围'].apply(lambda x:x.split(',')).values
print(x_cut)
qichacha_cut.head()

[list(['批发', '零售']) list(['加工', '销售'])
 list(['小吃', '服务', '依法', '须', '批准', '项目', '相关', '部门', '批准', '后方', '开展', '经营', '活动'])
 ...
 list(['茶叶', '茶具', '批发', '零售', '依法', '须', '批准', '项目', '相关', '部门', '批准', '后方', '开展', '经营', '活动'])
 list(['汽车', '维修', '保养', '钣', '金烤', '漆', '电脑', '解码', '依法', '须', '批准', '项目', '相关', '部门', '批准', '后方', '开展', '经营', '活动'])
 list(['藤器', '制品'])]


,公司名,经营范围,所在产业
0,刘向岗在业,"批发,零售",零售业
1,安丘市兴建新型装饰材料制品厂吊销,"加工,销售",NaN
2,安丘市浩红小吃铺存续,"小吃,服务,依法,须,批准,项目,相关,部门,批准,后方,开展,经营,活动",NaN
3,滨州市滨城区建利纸制品批发部存续,"纸制品,需,许可,经营,许可证,经营",NaN
4,滨州市滨城区倩倩化妆品店存续,化妆品,NaN


In [36]:
def getLDA(topic_num):
    from gensim.models import LdaMulticore
    from gensim.corpora import Dictionary
    # gensim
    dictionary = Dictionary(x_cut)
    print(dictionary)
    corpus = [dictionary.doc2bow(doc) for doc in x_cut]
    lda = LdaMulticore(corpus=corpus,  # LDA训练语料
                   id2word=dictionary, # id到单词的映射表
                   num_topics=topic_num)      # LDA主题数量 
    return lda,corpus,dictionary
topic_num = 5
lda,corpus,dictionary = getLDA(topic_num)
lda.print_topics()

Dictionary(1878 unique tokens: ['批发', '零售', '加工', '销售', '依法']...)


[(0,
  '0.067*"批准" + 0.041*"经营" + 0.035*"部门" + 0.035*"开展" + 0.034*"项目" + 0.034*"活动" + 0.033*"相关" + 0.033*"须" + 0.027*"依法" + 0.027*"后方"'),
 (1,
  '0.056*"批准" + 0.049*"经营" + 0.033*"须" + 0.032*"活动" + 0.030*"开展" + 0.030*"相关" + 0.029*"部门" + 0.029*"依法" + 0.028*"后方" + 0.026*"项目"'),
 (2,
  '0.038*"销售" + 0.037*"批准" + 0.034*"经营" + 0.024*"相关" + 0.022*"须" + 0.020*"依法" + 0.019*"活动" + 0.019*"开展" + 0.018*"后方" + 0.017*"项目"'),
 (3,
  '0.086*"批准" + 0.068*"经营" + 0.047*"项目" + 0.046*"须" + 0.044*"后方" + 0.044*"开展" + 0.044*"活动" + 0.044*"依法" + 0.042*"部门" + 0.041*"相关"'),
 (4,
  '0.030*"批准" + 0.023*"销售" + 0.020*"服务" + 0.019*"经营" + 0.016*"须" + 0.014*"许可证" + 0.014*"含" + 0.014*"开展" + 0.013*"食品" + 0.013*"相关"')]

In [31]:
def get_topic_distribute(lda,doc):
    return lda[doc]

lda_matrix = np.zeros((qichacha_cut.shape[0],topic_num),dtype='float64')
for index,doc in enumerate(corpus):
    topics = get_topic_distribute(lda,corpus[index])
    for topic_tuple in topics:
        lda_matrix[index,topic_tuple[0]] = topic_tuple[1]

[(0, 0.7329545), (1, 0.06686983), (2, 0.06683207), (3, 0.06667106), (4, 0.066672556)]
[0.73295498 0.0668694  0.06683198 0.06667105 0.06667256]
[(0, 0.7329545), (1, 0.06686985), (2, 0.06683209), (3, 0.06667106), (4, 0.066672556)]


In [34]:
# SVM训练
def train_svm(lda_matrix,label):
    clf = SVC()
    clf.fit(lda_matrix, label) 
    return clf

In [42]:
def predict(dictionary,description,clf):
    a = dictionary.doc2bow(jieba.lcut(description))
    topic_distribute = lda[a]
    res = clf.predict(topic_distribute)
    return res

[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]

### 补全地市

In [32]:
def get_area():
    df2 = pd.read_csv('qichacha.csv',index_col=0)
    return df2['登记机关'].apply(lambda x:x[:2])
    
qichacha_cut['所在区域']=get_area()
qichacha_cut.head()

,公司名,经营范围,所在产业,所在区域
0,刘向岗在业,"批发,零售",零售业,安丘
1,安丘市兴建新型装饰材料制品厂吊销,"加工,销售",NaN,安丘
2,安丘市浩红小吃铺存续,"小吃,服务,依法,须,批准,项目,相关,部门,批准,后方,开展,经营,活动",NaN,安丘
3,滨州市滨城区建利纸制品批发部存续,"纸制品,需,许可,经营,许可证,经营",NaN,滨州
4,滨州市滨城区倩倩化妆品店存续,化妆品,NaN,滨州


In [15]:
qichacha_cut.shape[0]

1150